In [1]:
import sys 
sys.path.append('../scripts/')
from kf import *   #誤差楕円を描くのに利用

In [2]:
def make_ax(): #axisの準備
    fig = plt.figure(figsize=(4,4))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlim(-5,5)                  
    ax.set_ylim(-5,5) 
    ax.set_xlabel("X",fontsize=10) 
    ax.set_ylabel("Y",fontsize=10)  
    return ax

def draw_trajectory(xs, ax): #軌跡の描画
    poses = [xs[s] for s in range(len(xs))]
    ax.scatter([e[0] for e in poses], [e[1] for e in poses], s=5, marker=".", color="black")
    ax.plot([e[0] for e in poses], [e[1] for e in poses], linewidth=0.5, color="black")
    
def draw_observations(xs, zlist, ax): #センサ値の描画
    for s in range(len(xs)):
        if s not in zlist:
            continue
            
        for obs in zlist[s]:
            x, y, theta = xs[s]
            ell, phi = obs[1][0], obs[1][1]
            mx = x + ell*math.cos(theta + phi)
            my = y + ell*math.sin(theta + phi)
            ax.plot([x,mx], [y,my], color="pink", alpha=0.5)
            
def draw_edges(edges, ax):
    for e in edges:
        ax.plot([e.x1[0], e.x2[0]], [e.x1[1] ,e.x2[1]], color="red", alpha=0.5)
        
def draw_landmarks(ms, ax): 
    ax.scatter([ms[k][0] for k in ms], [ms[k][1] for k in ms], s=100, marker="*", color="blue", zorder=100)
    
def draw(xs, zlist, edges, ms={}):  #ms追加
    ax = make_ax()
    draw_observations(xs, zlist, ax)
    draw_trajectory(xs, ax)
    draw_landmarks(ms, ax)  #追加
    plt.show()

In [3]:
def read_data(): ###graphbasedslam_2d_sensor_readdata
    hat_xs = {} 
    zlist = {} 
    delta = 0.0
    us = {}

    with open("log2.txt") as f: #log2.txtに変えておく
        for line in f.readlines():
            tmp = line.rstrip().split()

            step = int(tmp[1])
            if tmp[0] == "x": 
                hat_xs[step] = np.array([float(tmp[2]), float(tmp[3]), float(tmp[4])]).T
            elif tmp[0] == "z": 
                if step not in zlist: 
                    zlist[step] = []
                zlist[step].append((int(tmp[2]), np.array([float(tmp[3]), float(tmp[4])]).T)) #変更。ψを読み込まないように
            elif tmp[0] == "delta": 
                delta = float(tmp[1])
            elif tmp[0] == "u":
                us[step] = np.array([float(tmp[2]), float(tmp[3])]).T 
                
        return hat_xs, zlist, us, delta #us, deltaも返す

In [4]:
class ObsEdge:   ###graphbasedslam_2d_sensor_obsedge
    def __init__(self, t1, t2, z1, z2, xs, sensor_noise_rate=[0.14, 0.05]):  #ψの標準偏差を削除
        assert z1[0] == z2[0] 

        self.t1, self.t2 = t1, t2  
        self.x1, self.x2 = xs[t1], xs[t2]
        self.z1, self.z2 = z1[1], z2[1]
        
        s1 = math.sin(self.x1[2] + self.z1[1]) 
        c1 = math.cos(self.x1[2] + self.z1[1])
        s2 = math.sin(self.x2[2] + self.z2[1])
        c2 = math.cos(self.x2[2] + self.z2[1])

        ##誤差の計算##
        hat_e = self.x2[0:2] - self.x1[0:2] + np.array([       #self.x2とself.x1は上の2行だけを使う
            self.z2[0]*c2 - self.z1[0]*c1, 
            self.z2[0]*s2 - self.z1[0]*s1
        ])                                                                   #ψに関する行列の行と、正規化していた行を削除。
            
        ##精度行列の作成## 
        Q1 = np.diag([(self.z1[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2]) #ψの分散を削除
        R1 = - np.array([[c1, -self.z1[0]*s1],
                                     [s1,  self.z1[0]*c1]])    #3行目、3列目を削除
        
        Q2 = np.diag([(self.z2[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2]) #ψの分散を削除
        R2 = np.array([[c2, -self.z2[0]*s2],
                                  [s2,  self.z2[0]*c2]])    #3行目、3列目を削除
        
        Sigma = R1.dot(Q1).dot(R1.T) + R2.dot(Q2).dot(R2.T)
        Omega = np.linalg.inv(Sigma)                                                  #2x2行列になる
        
        ##大きな精度行列と係数ベクトルの各部分を計算##
        B1 = - np.array([[1, 0, -self.z1[0]*s1],
                                    [0, 1, self.z1[0]*c1]])                                  #3行目を削除
        B2 = np.array([[1, 0,  -self.z2[0]*s2],
                                   [0, 1,   self.z2[0]*c2]])                                  #3行目を削除
        
        self.omega_upperleft = B1.T.dot(Omega).dot(B1)         #ここは計算すると3x3行列のままになる
        self.omega_upperright = B1.T.dot(Omega).dot(B2)
        self.omega_bottomleft = B2.T.dot(Omega).dot(B1)
        self.omega_bottomright = B2.T.dot(Omega).dot(B2)
        
        self.xi_upper = - B1.T.dot(Omega).dot(hat_e)         #ここも計算すると3次元縦ベクトルのままになる
        self.xi_bottom = - B2.T.dot(Omega).dot(hat_e)

In [5]:
class MotionEdge:
    def __init__(self, t1, t2, xs, us, delta, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}):
        self.t1, self.t2 = t1, t2                   #時刻の記録
        self.hat_x1, self.hat_x2 = xs[t1], xs[t2]    #各時刻の姿勢

        nu, omega = us[t2]
        if abs(omega) < 1e-5: omega = 1e-5 #ゼロにすると式が変わるので避ける

        M = matM(nu, omega, delta, motion_noise_stds)
        A = matA(nu, omega, delta, self.hat_x1[2])
        F = matF(nu, omega, delta, self.hat_x1[2])
        
        self.Omega = np.linalg.inv(A.dot(M).dot(A.T) + np.eye(3)*0.0001) #標準偏差0.01の雑音を足す
        
        self.omega_upperleft = F.T.dot(self.Omega).dot(F)
        self.omega_upperright = -F.T.dot(self.Omega)
        self.omega_bottomleft = - self.Omega.dot(F)
        self.omega_bottomright = self.Omega
        
        x2 = IdealRobot.state_transition(nu, omega, delta, self.hat_x1)
        self.xi_upper = F.T.dot(self.Omega).dot(self.hat_x2 - x2)
        self.xi_bottom = -self.Omega.dot(self.hat_x2 - x2)

In [6]:
import itertools 
def make_edges(hat_xs, zlist):
    landmark_keys_zlist = {}

    for step in zlist: 
        for z in zlist[step]:
            landmark_id = z[0]
            if landmark_id not in landmark_keys_zlist: 
                landmark_keys_zlist[landmark_id] = []

            landmark_keys_zlist[landmark_id].append((step, z))
    
    edges = []
    for landmark_id in landmark_keys_zlist:
        step_pairs = list(itertools.combinations(landmark_keys_zlist[landmark_id], 2))
        edges += [ObsEdge(xz1[0], xz2[0], xz1[1], xz2[1], hat_xs) for xz1, xz2 in step_pairs]
        
    return edges, landmark_keys_zlist #ランドマークをキーにしたリストlandmark_keys_zlistも返す

In [7]:
def add_edge(edge, Omega, xi): 
    f1, f2 = edge.t1*3, edge.t2*3
    t1 ,t2 = f1 + 3, f2 + 3
    Omega[f1:t1, f1:t1] += edge.omega_upperleft
    Omega[f1:t1, f2:t2] += edge.omega_upperright
    Omega[f2:t2, f1:t1] += edge.omega_bottomleft
    Omega[f2:t2, f2:t2] += edge.omega_bottomright
    xi[f1:t1] += edge.xi_upper
    xi[f2:t2] += edge.xi_bottom

In [8]:
hat_xs, zlist, us, delta = read_data()  
dim = len(hat_xs)*3

for n in range(1, 10000): 
    ##エッジ、大きな精度行列、係数ベクトルの作成##
    edges, _ = make_edges(hat_xs, zlist)  #返す変数が2つになるので「_」で合わせる

    for i in range(len(hat_xs)-1): #行動エッジの追加
        edges.append(MotionEdge(i, i+1, hat_xs, us, delta))
        
    Omega = np.zeros((dim, dim))
    xi = np.zeros(dim)
    Omega[0:3, 0:3] += np.eye(3)*1000000

    ##軌跡を動かす量（差分）の計算##
    for e in edges:
        add_edge(e, Omega, xi) 

    delta_xs = np.linalg.inv(Omega).dot(xi) 
    
    ##推定値の更新##
    for i in range(len(hat_xs)):
        hat_xs[i] += delta_xs[i*3:(i+1)*3] 
        
    ##終了判定##
    diff = np.linalg.norm(delta_xs) 
    print("{}回目の繰り返し: {}".format(n, diff))
    if diff < 0.02:
        draw(hat_xs, zlist, edges)
        break

1回目の繰り返し: 13.339392352057338
2回目の繰り返し: 7.661718879214364
3回目の繰り返し: 3.397419760080383
4回目の繰り返し: 3.6312169310226174
5回目の繰り返し: 2.8579570101141036
6回目の繰り返し: 2.6648824513716205
7回目の繰り返し: 2.3632499162121037
8回目の繰り返し: 2.1744191947821734
9回目の繰り返し: 2.000489916210123
10回目の繰り返し: 1.8635834359128987
11回目の繰り返し: 1.7467203897693486
12回目の繰り返し: 1.6490794291884514
13回目の繰り返し: 1.5659097978082592
14回目の繰り返し: 1.4950362495254923
15回目の繰り返し: 1.4342774381200105
16回目の繰り返し: 1.382101458501944
17回目の繰り返し: 1.3372379589502619
18回目の繰り返し: 1.298693562617502
19回目の繰り返し: 1.2656619882668807
20回目の繰り返し: 1.2374924044185334
21回目の繰り返し: 1.2136539760189065
22回目の繰り返し: 1.1937131428369847
23回目の繰り返し: 1.1773149373214282
24回目の繰り返し: 1.16416909307157
25回目の繰り返し: 1.1540391402125711
26回目の繰り返し: 1.1467339787319084
27回目の繰り返し: 1.1421012902902652
28回目の繰り返し: 1.140022408618522
29回目の繰り返し: 1.1404083367113662
30回目の繰り返し: 1.1431966888162113
31回目の繰り返し: 1.148349388188342
32回目の繰り返し: 1.1558509956430336
33回目の繰り返し: 1.1657075759372544
34回目の繰り返し: 1.177946033898186

271回目の繰り返し: 0.07682087822606846
272回目の繰り返し: 0.07479858600763571
273回目の繰り返し: 0.07284102192296416
274回目の繰り返し: 0.07094593927129311
275回目の繰り返し: 0.06911118101915194
276回目の繰り返し: 0.06733467545875722
277回目の繰り返し: 0.06561443212528616
278回目の繰り返し: 0.06394853795319445
279回目の繰り返し: 0.0623351536559069
280回目の繰り返し: 0.0607725103127657
281回目の繰り返し: 0.05925890614986541
282回目の繰り返し: 0.05779270349986104
283回目の繰り返し: 0.05637232593262126
284回目の繰り返し: 0.054996255539852205
285回目の繰り返し: 0.05366303036846474
286回目の繰り返し: 0.05237124199204993
287回目の繰り返し: 0.051119533207019514
288回目の繰り返し: 0.0499065958537849
289回目の繰り返し: 0.0487311687471473
290回目の繰り返し: 0.047592035714203326
291回目の繰り返し: 0.046488023730174854
292回目の繰り返し: 0.04541800114824676
293回目の繰り返し: 0.04438087601731754
294回目の繰り返し: 0.04337559448010011
295回目の繰り返し: 0.04240113925120284
296回目の繰り返し: 0.0414565281660409
297回目の繰り返し: 0.04054081279980492
298回目の繰り返し: 0.03965307715005564
299回目の繰り返し: 0.03879243638179578
300回目の繰り返し: 0.03795803562875125
301回目の繰り返し: 0.03714904885065966
302回目の繰り返

<IPython.core.display.Javascript object>

In [9]:
_, zlist_landmark = make_edges(hat_xs, zlist) 
zlist_landmark

{1: [(0, (1, array([2.06692697, 0.1072544 ]))),
  (1, (1, array([ 1.5096788 , -0.24896954]))),
  (2, (1, array([ 0.70599846, -0.87980216]))),
  (19, (1, array([5.88825157, 0.91219263]))),
  (20, (1, array([5.49530319, 0.77184918]))),
  (21, (1, array([4.22159569, 0.45413407]))),
  (22, (1, array([3.30074879, 0.25687774]))),
  (23, (1, array([3.81608544, 0.0631331 ]))),
  (24, (1, array([ 2.43059288, -0.0623398 ]))),
  (25, (1, array([ 1.78897471, -0.50307633]))),
  (45, (1, array([4.77636009, 0.71157766]))),
  (46, (1, array([5.66554641, 0.58590108]))),
  (47, (1, array([5.09707481, 0.35610579]))),
  (48, (1, array([3.69124295, 0.27880156]))),
  (49, (1, array([3.33045759, 0.13861935]))),
  (50, (1, array([ 2.38280131, -0.10618313]))),
  (51, (1, array([ 1.30205637, -0.54669782])))],
 2: [(2, (2, array([6.35152367, 0.92637676]))),
  (3, (2, array([6.00363588, 0.69310826]))),
  (4, (2, array([4.65862126, 0.51401244]))),
  (5, (2, array([6.12674004, 0.41674825]))),
  (7, (2, array([3.016

In [10]:
class MapEdge: ###graphbasedslam_2d_sensor_mapedge
    def __init__(self, t, z, head_t, head_z, xs, sensor_noise_rate=[0.14, 0.05]):  #センサの雑音モデルを削除
        self.x = xs[t]
        self.z = z
        
        self.m = self.x[0:2] + np.array([z[0]*math.cos(self.x[2] + z[1]), z[0]*math.sin(self.x[2] + z[1])]).T  #3行目削除
            
        ##精度行列の計算## 
        Q1 = np.diag([(self.z[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2]) #ψの分散を削除
        
        s1 = math.sin(self.x[2] + self.z[1]) 
        c1 = math.cos(self.x[2] + self.z[1])
        R = np.array([[-c1, self.z[0]*s1],
                                [-s1,-self.z[0]*c1]]) #3行目、3列目を削除
        
        self.Omega = np.linalg.inv(R.dot(Q1).dot(R.T)) #2x2行列になる
        self.xi = self.Omega.dot(self.m)   #2次元ベクトルになる

In [11]:
ms = {}###graphbasedslam_2d_sensor_mapexec
for landmark_id in zlist_landmark:
    edges = []
    head_z = zlist_landmark[landmark_id][0]
    for z in zlist_landmark[landmark_id]:
        edges.append(MapEdge(z[0], z[1][1], head_z[0], head_z[1][1], hat_xs))
        
    Omega = np.zeros((2,2)) #2x2に
    xi = np.zeros(2)                 #2次元に
    for e in edges:
        Omega += e.Omega
        xi += e.xi
        
    ms[landmark_id] = np.mean([e.m for e in edges], axis=0)
    
draw(hat_xs, zlist, edges, ms)

<IPython.core.display.Javascript object>

In [12]:
actual_pos = [(-4,2), (2,-3), (3,3), (0,4), (1,1), (-3,-1)]
ms

{1: array([ 1.7180129 , -3.08128803]),
 2: array([ 7.1059205 , -5.44003902]),
 3: array([ 8.92807734, -2.54635957]),
 0: array([7.81897946, 1.66669421]),
 5: array([4.49762136, 1.38200136])}

In [13]:
from decimal import Decimal, ROUND_HALF_UP
def distance(landmark_id, ref_id1, ref_id2, ms):
    m, ref_m1, ref_m2 = ms[landmark_id], ms[ref_id1], ms[ref_id2]
    d = math.sqrt( (m[0]-ref_m1[0])**2 + (m[1]-ref_m1[1])**2 )
    ang = math.atan2(m[1] - ref_m1[1], m[0] - ref_m1[0]) - math.atan2(ref_m2[1] - ref_m1[1], ref_m2[0] - ref_m1[0])
    return Decimal(d).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP), Decimal(ang/math.pi*180).quantize(Decimal("0.1"), rounding=ROUND_HALF_UP)

In [14]:
for i in range(1,6):
    d, ang = distance(i, 0, 1, ms)
    true_d, true_ang = distance(i, 0, 1, actual_pos)
    print(i, true_d, true_ang, d, ang, d - true_d, ang - true_ang)

1 7.81 0.0 7.73 0.0 -0.08 0.0
2 7.07 47.9 7.14 46.4 0.07 -1.5
3 4.47 66.4 4.36 66.9 -0.11 0.5


KeyError: 4